In [1]:
import sys
import os

# --- 1. SETUP PATHS ---
# Get the project root by going up one level from 'notebooks/'
current_dir = os.getcwd()
project_root = os.path.abspath(os.path.join(current_dir, '..'))
src_path = os.path.join(project_root, 'src')

# Add 'src' to system path so we can import our custom files
if src_path not in sys.path:
    sys.path.append(src_path)
    print(f"✅ Added 'src' to Python path: {src_path}")

# --- 2. STANDARD IMPORTS ---
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np

# --- 3. CUSTOM IMPORTS ---
# These will now work because of the path fix above
from model_hqcnn_robust import HQCNN_Residual
from data_loader_robust import get_robust_loaders, get_ctsd_loader, AddGaussianNoise

print("🚀 Libraries Loaded Successfully!")

✅ Added 'src' to Python path: /home/akash_kishore/HQCNN_Project/src
🚀 Libraries Loaded Successfully!


In [3]:
# --- CONFIGURATION ---
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define Data Paths (Relative to project root)
# NOTE: Check if your 'data' folder is in the project root
DATA_DIR_GTSB = os.path.join(project_root, 'data/GTSRB')      
DATA_DIR_CTSD = os.path.join(project_root, 'data/CTSD') 

# Hyperparameters
BATCH_SIZE = 32
LR = 0.0001           # Low LR for stability
EPOCHS = 10           
NOISE_LEVEL = 0.15    # Train with 15% noise to force robustness

print(f"⚙️  Running on: {DEVICE}")
print(f"📂 Reading GTSB Data from: {DATA_DIR_GTSB}")

# --- LOAD DATA ---
try:
    print("⏳ Loading Robust Data Loaders...")
    train_loader, val_loader = get_robust_loaders(DATA_DIR_GTSB, BATCH_SIZE, noise_level=NOISE_LEVEL)
    print(f"✅ Data Ready: {len(train_loader.dataset)} Training Samples | {len(val_loader.dataset)} Validation Samples")
except Exception as e:
    print(f"❌ Error loading data: {e}")
    print("Double check your DATA_DIR_GTSB path!")

⚙️  Running on: cuda
📂 Reading GTSB Data from: /home/akash_kishore/HQCNN_Project/data/GTSRB
⏳ Loading Robust Data Loaders...
✅ Data Ready: 33327 Training Samples | 5882 Validation Samples


In [4]:
# 1. INITIALIZE MODEL
model = HQCNN_Residual(n_classes=43).to(DEVICE)

# 2. OPTIMIZER & LOSS
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LR)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=4, gamma=0.1)

# 3. TRAINING LOOP
best_acc = 0.0
history = {'train_loss': [], 'val_acc': []}

print(f"🧠 Starting Training for {EPOCHS} epochs...")

for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0
    
    # Progress Bar for Training
    loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}")
    
    for inputs, labels in loop:
        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        loop.set_postfix(loss=loss.item())
        
    scheduler.step()
    
    # Validation Phase (Clean Accuracy)
    model.eval()
    correct = 0; total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
    val_acc = 100 * correct / total
    history['train_loss'].append(running_loss / len(train_loader))
    history['val_acc'].append(val_acc)
    
    print(f"   Epoch {epoch+1} Result: Clean Validation Accuracy = {val_acc:.2f}%")
    
    # Save Best Model
    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), 'best_hqcnn_robust.pth')
        print("   >>> ✅ New Best Model Saved (best_hqcnn_robust.pth)")

print(f"\n🎉 Training Complete. Best Clean Accuracy: {best_acc:.2f}%")

/home/akash_kishore/miniconda3/envs/hqcnn_env/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/akash_kishore/miniconda3/envs/hqcnn_env/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


🧠 Starting Training for 10 epochs...


Epoch 1/10: 100%|██████████| 1042/1042 [02:58<00:00,  5.85it/s, loss=0.262] 


   Epoch 1 Result: Clean Validation Accuracy = 98.37%
   >>> ✅ New Best Model Saved (best_hqcnn_robust.pth)


Epoch 2/10: 100%|██████████| 1042/1042 [02:27<00:00,  7.07it/s, loss=0.0757]


...

   Epoch 8 Result: Clean Validation Accuracy = 99.74%
   >>> ✅ New Best Model Saved (best_hqcnn_robust.pth)


Epoch 9/10: 100%|██████████| 1042/1042 [02:27<00:00,  7.09it/s, loss=0.186]  


   Epoch 9 Result: Clean Validation Accuracy = 99.64%


Epoch 10/10: 100%|██████████| 1042/1042 [02:24<00:00,  7.20it/s, loss=0.058]  


   Epoch 10 Result: Clean Validation Accuracy = 99.71%

🎉 Training Complete. Best Clean Accuracy: 99.74%
